In [2]:
from model import get_model

model, tokenizer = get_model('Qwen/Qwen2.5-1.5B-Instruct')

c:\Users\User\OneDrive\Desktop\work\peft-learn\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [4]:
from datasets import load_from_disk
import os

In [23]:
ds = load_from_disk(os.path.join('data/banking77/processed_v1/train'))


In [58]:
samples = ds[:10]['text']

In [66]:
texts =  [
        tokenizer.apply_chat_template(
        # messages,
        [
            {"role": "system", "content": "You are a support triage assistant.\nReturn STRICT JSON with keys: category, priority, first_response_steps (array of strings).\nTicket:"},
            {"role": "user", "content": sample.split('\n')[2].split(' ', 1)[1]}
        ],
        tokenize=False,
        add_generation_prompt=True,
    )
    for sample in samples
]

model_inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, padding_side='left').to(model.device)

In [67]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

In [69]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
response

['{\n  "category": "Billing",\n  "priority": "High",\n  "first_response_steps": [\n    "Please provide your billing details to expedite the processing.",\n    "Ensure you have provided all necessary information and documents for your request."\n  ]\n}',
 'category: Support Tickets\n\npriority: High\n\nfirst_response_steps:\n- Check your mail for any notifications about the order status or tracking number provided by the merchant.\n- Contact the merchant directly to inform them that you haven’t received your card within 2 weeks and ask them to resend it or provide an alternative solution.\n- If there is no response from the merchant, consider reaching out to customer service of the shipping company they use to track your package.',
 '{\n  "category": "Support",\n  "priority": "High",\n  "first_response_steps": [\n    "Check your account for any updates or messages from the service provider.",\n    "If you haven\'t received an update in a few days, consider contacting customer support di

In [74]:
i = 0
for res in response:
    print(f'------------{samples[i].split('\n')[2].split(' ', 1)[1]}')
    print(f'------------{res}')
    i+=1

------------I am still waiting on my card?
------------{
  "category": "Billing",
  "priority": "High",
  "first_response_steps": [
    "Please provide your billing details to expedite the processing.",
    "Ensure you have provided all necessary information and documents for your request."
  ]
}
------------What can I do if my card still hasn't arrived after 2 weeks?
------------category: Support Tickets

priority: High

first_response_steps:
- Check your mail for any notifications about the order status or tracking number provided by the merchant.
- Contact the merchant directly to inform them that you haven’t received your card within 2 weeks and ask them to resend it or provide an alternative solution.
- If there is no response from the merchant, consider reaching out to customer service of the shipping company they use to track your package.
------------I have been waiting over a week. Is the card still coming?
------------{
  "category": "Support",
  "priority": "High",
  "first_

In [89]:
test_ds = load_from_disk(os.path.join('data/banking77/processed_v1/test'))